In [23]:
#! pip install google-cloud-storage

# Required Libraries and Setup config

In [24]:
import pandas as pd
from datetime import datetime
from google.cloud import storage
import subprocess
import os
import getpass


timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
environment = 'prod'

# bucket information 
bucket_name = 'gmb-database-backup'
backup_destination_path = f'{environment}/backups/{timestamp}/'

username = 'gmb'
public_ip_address = '34.133.118.52'
db_name = 'gmb'
port = '5432'
password = getpass.getpass('Enter password:')


# postgresql configuration
db_name = db_name
username = username
host = public_ip_address
port = port


Enter password:········


### Create a Bucket

In [25]:
def create_bucket():
    storage_client = storage.Client()
    
    if not storage_client.lookup_bucket(bucket_name):
        bucket = storage_client.create_bucket(bucket_name)
        print(f"Bucket {bucket_name} created.")
    else:
        print(f"Bucket {bucket_name} already exists.")

### Create the Database Backup

In [26]:
def create_backup():
    
    pg_dump_path = "/System/Volumes/Data/Library/PostgreSQL/15/bin/pg_dump"
    backup_fname = 'prod_backup.sql'
    command = f'{pg_dump_path} -U {username} -h {host} -p {port} {db_name} > {backup_fname}'
    
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode != 0:
        print("Error creating backup:", result.stderr.decode())
        return None
    
    return backup_fname


### Upload the Backup to GCS

In [27]:
def upload_to_gcs(file_path):
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(os.path.join(backup_destination_path, os.path.basename(file_path)))
    
    blob.upload_from_filename(file_path)
    
    print(f"File {file_path} uploaded to {bucket_name}/{backup_destination_path}.")

In [28]:
backup_file = create_backup()
if backup_file:
    upload_to_gcs(backup_file)

KeyboardInterrupt: 